In [42]:
import pandas as pd
import re
import nltk
nltk.download('wordnet')
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
# from textblob import TextBlob

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Quang\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Quang\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [43]:
data = pd.read_csv("Train.csv", sep=',')
test = pd.read_csv("Test.csv", sep=',')

In [44]:
stopwordlist = ['a', 'about', 'above', 'after', 'again', 'ain', 'all', 'am', 'an',
             'and','any','are', 'as', 'at', 'be', 'because', 'been', 'before',
             'being', 'below', 'between','both', 'by', 'can', 'd', 'did', 'do',
             'does', 'doing', 'down', 'during', 'each','few', 'for', 'from',
             'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here',
             'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in',
             'into','is', 'it', 'its', 'itself', 'just', 'll', 'm', 'ma',
             'me', 'more', 'most','my', 'myself', 'now', 'o', 'of', 'on', 'once',
             'only', 'or', 'other', 'our', 'ours','ourselves', 'out', 'own', 're','s',
              'same', 'she', "shes", 'should', "shouldve",'so', 'some', 'such',
             't', 'than', 'that', "thatll", 'the', 'their', 'theirs', 'them',
             'themselves', 'then', 'there', 'these', 'they', 'this', 'those',
             'through', 'to', 'too','under', 'until', 'up', 've', 'very', 'was',
             'we', 'were', 'what', 'when', 'where','which','while', 'who', 'whom',
             'why', 'will', 'with', 'y', 'you', "youd","youll", "youre",
             "youve", 'your', 'yours', 'yourself', 'yourselves']
STOPWORDS = set(stopwordlist)

In [45]:
tweet = 'This is happiness cute. the about tacos ran crying #thisisus @nbcthisisus https://t.co/ndxqyl4gjk AKJSBG !!! on !! ???? 1029468798346y 368258 .,/, $#%$'
def preprocess_string(str):
    lemmatizer = WordNetLemmatizer()

    str = str.lower()
    str = re.sub(r'@[^\s]+','',str)
    str = re.sub(r'((www.[^s]+)|(https?://[^\s]+))',' ',str)
    str = re.sub(r'[^a-zA-Z?!#S]',' ',str)
    str = re.sub(r'\b\w{1,2}\b', '', str)
    str = re.sub(r'!', ' ! ', str)
    str = re.sub(r'\?', ' ? ', str)
    str = " ".join([word for word in str.split() if word not in STOPWORDS])
    str = str.split()
    # blob_object = TextBlob(str)
    # str = blob_object.words (this removes all puncuations)

    str = [lemmatizer.lemmatize(x) for x in str]
    str = ' '.join(str)
    return str

print(preprocess_string(tweet))

data['text'] = data['text'].apply(lambda text: preprocess_string(text))
test['text'] = data['text'].apply(lambda text: preprocess_string(text))
data.to_csv('preprocessed-train-data.csv', index=False)

X_train_raw = data.text
y_train = data.sentiment

X_test_raw = test.text
X_test_id = test.id



happiness cute taco ran cry #thisisus akjsbg ! ! ! ! ! ? ? ? ? #


BOW

In [49]:
from sklearn.feature_extraction.text import CountVectorizer

BoW_vectorizer = CountVectorizer(max_df=0.90, min_df=2)

#Build the feature set (vocabulary) and vectorise the Tarin dataset using BoW
X_train_BoW = BoW_vectorizer.fit_transform(X_train_raw)

#Use the feature set (vocabulary) from Train to vectorise the Test dataset 
X_test_BoW = BoW_vectorizer.transform(X_test_raw)

print("Train feature space size (using BoW):",X_train_BoW.shape)
print("Test feature space size (using BoW):",X_test_BoW.shape)

output_dict = BoW_vectorizer.vocabulary_
output_pd = pd.DataFrame(list(output_dict.items()),columns = ['word','count'])

output_pd.T.to_csv('BoW-vocab.csv',index=False)

Train feature space size (using BoW): (21802, 10410)
Test feature space size (using BoW): (6099, 10410)


In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()

#Build the feature set (vocabulary) and vectorise the Tarin dataset using TFIDF
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_raw)

#Use the feature set (vocabulary) from Train to vectorise the Test dataset 
X_test_tfidf = tfidf_vectorizer.transform(X_test_raw)

print("Train feature space size (using TFIDF):",X_train_tfidf.shape)
print("Test feature space size (using TFIDF):",X_test_tfidf.shape)

Train feature space size (using TFIDF): (21802, 23576)
Test feature space size (using TFIDF): (6099, 23576)


In [48]:
from sklearn.svm import LinearSVC
SVMmodel = LinearSVC()

SVMmodel.fit(X_train_tfidf, y_train)
y_pred_svm_tfidf = SVMmodel.predict(X_test_tfidf)

data = {'id':X_test_id,'text':X_test_raw,'sentiment':y_pred_svm_tfidf}
predictions = pd.DataFrame(data=data)
(predictions.drop(['text'], axis=1)).to_csv('svm-model-tfidf.csv', index=False)

SVMmodel.fit(X_train_BoW, y_train)
y_pred_svm_BoW = SVMmodel.predict(X_test_BoW)

data = {'id':X_test_id,'text':X_test_raw,'sentiment':y_pred_svm_BoW}
predictions = pd.DataFrame(data=data)
(predictions.drop(['text'], axis=1)).to_csv('svm-model-bow.csv', index=False)